## Optimizers

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

# Prepare data
X, y = make_moons(n_samples=2000, noise=0.2, random_state=0)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
ds = TensorDataset(X, y)
loader = DataLoader(ds, batch_size=128, shuffle=True)

# Small MLP
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
    def forward(self, x): return self.net(x)

def train_optimizer(opt_name, lr=1e-3, epochs=60):
    torch.manual_seed(0)
    model = MLP()
    if opt_name=='sgd':
        opt = optim.SGD(model.parameters(), lr=lr)
    elif opt_name=='sgd_mom':
        opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif opt_name=='adam':
        opt = optim.Adam(model.parameters(), lr=lr)
    elif opt_name=='adamw':
        opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    else:
        raise ValueError(opt_name)
    loss_fn = nn.CrossEntropyLoss()
    sched_step = lr_sched.StepLR(opt, step_size=20, gamma=0.1)
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0.0
        for xb, yb in loader:
            opt.zero_grad()
            out = model(xb)
            loss = loss_fn(out, yb)
            loss.backward()
            opt.step()

            sched_step.step()
            epoch_loss += loss.item()
        losses.append(epoch_loss/len(loader))
    return losses

## Gradient Norms & Gradient Clipping

In [ ]:
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


In [ ]:
torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=0.5)


## Learning Rate Schedules

In [ ]:
import torch.optim.lr_scheduler as lr_sched

sched_step = lr_sched.StepLR(opt, step_size=20, gamma=0.1)
sched_multi = lr_sched.MultiStepLR(opt, milestones=[30, 60, 90], gamma=0.1)
sched_plateau = lr_sched.ReduceLROnPlateau(opt, factor=0.5, patience=5)
sched_cosine = lr_sched.CosineAnnealingLR(opt, T_max=50)
sched_warmup_cosine = lr_sched.CosineAnnealingWarmRestarts(opt, T_0=10)


## Mixed Precision (AMP)

In [ ]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

with autocast():
    output = model(x)
    loss = loss_fn(output, y)

scaler.scale(loss).backward()
scaler.step(opt)
scaler.update()


## Data Augmentation

In [ ]:
from torchvision import transforms

augment_basic = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
])

augment_color = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
])

augment_geometric = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, translate=(0.1,0.1)),
])

augment_strong = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomApply([transforms.ColorJitter()], p=0.8),
    transforms.RandomGrayscale(p=0.2),
])


## Weight Decay

In [ ]:
opt = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)  # strong
opt = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)  # moderate
opt = optim.SGD(model.parameters(), lr=0.01, weight_decay=5e-4)   # typical for CNNs


## Early Stopping

In [ ]:
best_loss = float('inf')
patience = 10
counter = 0

for epoch in range(epochs):
    train(...)
    val_loss = validate(...)

    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best.pt")
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered.")
        break


## Exercise:
Reach strong accuracy on CIFAR-10 by applying the techniques (data augmentation, optimizer selection, weight decay, schedulers, gradient clipping, AMP, Early Stopping).

In [1]:
# ==========================================
# 1️⃣ نصب و ایمپورت کتابخانه‌ها
# ==========================================
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader

# ==========================================
# 2️⃣ انتخاب دستگاه (GPU اگر موجود باشد)
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ==========================================
# 3️⃣ Data Augmentation و آماده‌سازی داده‌ها
# ==========================================
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# ==========================================
# 4️⃣ دانلود و آماده‌سازی دیتاست CIFAR-10
# ==========================================
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=test_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=2
)

# ==========================================
# 5️⃣ مدل (ResNet18)
# ==========================================
model = torchvision.models.resnet18(num_classes=10)
model = model.to(device)

# ==========================================
# 6️⃣ Loss Function و Optimizer + Weight Decay
# ==========================================
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(
    model.parameters(),
    lr=0.001,
    weight_decay=0.01
)

# ==========================================
# 7️⃣ Scheduler (کنترل Learning Rate)
# ==========================================
scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=10,
    gamma=0.5
)

# ==========================================
# 8️⃣ AMP (Mixed Precision)
# ==========================================
scaler = GradScaler()

# ==========================================
# 9️⃣ Early Stopping تنظیمات
# ==========================================
best_accuracy = 0.0
patience = 5
no_improve_epochs = 0
epochs = 40

# ==========================================
# 🔟 حلقه آموزش و ارزیابی
# ==========================================
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Mixed Precision
        with autocast(enabled=(device.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        # Gradient Clipping
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    scheduler.step()

    # ==================================================
    # ارزیابی مدل روی Test Set
    # ==================================================
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total
    avg_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

    # ==================================================
    # Early Stopping
    # ==================================================
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_cifar10_model.pth")
        no_improve_epochs = 0
        print(">> New best model saved")
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= patience:
        print(">> Early stopping activated")
        break

print("Training finished")
print(f"Best Accuracy: {best_accuracy:.2f}%")


Using device: cuda


100%|██████████| 170M/170M [00:04<00:00, 39.6MB/s]
/tmp/ipython-input-1202253461.py:94: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-1202253461.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type == "cuda")):


Epoch [1/40] | Loss: 1.5717 | Accuracy: 53.96%
>> New best model saved
Epoch [2/40] | Loss: 1.2383 | Accuracy: 60.13%
>> New best model saved
Epoch [3/40] | Loss: 1.0884 | Accuracy: 64.69%
>> New best model saved
Epoch [4/40] | Loss: 0.9823 | Accuracy: 67.89%
>> New best model saved
Epoch [5/40] | Loss: 0.9016 | Accuracy: 68.93%
>> New best model saved
Epoch [6/40] | Loss: 0.8377 | Accuracy: 72.22%
>> New best model saved
Epoch [7/40] | Loss: 0.7973 | Accuracy: 73.89%
>> New best model saved
Epoch [8/40] | Loss: 0.7592 | Accuracy: 74.61%
>> New best model saved
Epoch [9/40] | Loss: 0.7301 | Accuracy: 76.60%
>> New best model saved
Epoch [10/40] | Loss: 0.6960 | Accuracy: 76.88%
>> New best model saved
Epoch [11/40] | Loss: 0.6119 | Accuracy: 79.16%
>> New best model saved
Epoch [12/40] | Loss: 0.5838 | Accuracy: 80.27%
>> New best model saved
Epoch [13/40] | Loss: 0.5718 | Accuracy: 80.46%
>> New best model saved
Epoch [14/40] | Loss: 0.5570 | Accuracy: 80.69%
>> New best model saved
E